# Module 03: Docker Desktop Installation

**Difficulty**: ⭐⭐  
**Estimated Time**: 60 minutes  
**Prerequisites**: 
- Module 01: Installation Methods Comparison
- Windows Pro, Enterprise, or Education edition
- Administrator access
- Virtualization enabled in BIOS

## Learning Objectives

By the end of this notebook, you will be able to:

1. Install and configure Docker Desktop on Windows with WSL2 backend
2. Create Docker volumes for persistent n8n data storage
3. Run n8n using docker run commands with proper configuration
4. Set up Docker Compose for declarative n8n deployment
5. Troubleshoot common Docker installation issues on Windows

## Why Docker Desktop for n8n?

Docker provides **production-ready** isolation that npm installations cannot match:

### Benefits
- **Isolation**: n8n runs in its own container, not affecting Windows
- **Version Control**: Easy to switch between n8n versions
- **Rollback**: Revert to previous version in seconds
- **Consistency**: Same environment on any machine
- **Auto-restart**: Containers automatically restart on failure

### Tradeoffs
- **Resource Overhead**: 2-4GB RAM for Docker Desktop + WSL2
- **Complexity**: More moving parts than npm
- **Windows Edition**: Requires Pro/Enterprise/Education (NOT Home)
- **Startup Time**: ~25 seconds vs 3 seconds for npm

**Bottom line**: If you need reliability and have Windows Pro+, Docker is worth the overhead.

## Prerequisites Verification

In [ ]:
import subprocess
import sys
import platform

def check_windows_edition():
    """
    Check if Windows edition supports Docker Desktop.
    Docker Desktop requires Pro, Enterprise, or Education.
    """
    try:
        result = subprocess.run(
            ['systeminfo'],
            capture_output=True,
            text=True,
            timeout=15
        )
        
        for line in result.stdout.split('\n'):
            if 'OS Name' in line:
                os_name = line.split(':')[1].strip()
                
                # Check for supported editions
                supported = any(edition in os_name for edition in 
                              ['Pro', 'Professional', 'Enterprise', 'Education'])
                
                return os_name, supported
        
        return "Unknown", False
    except Exception as e:
        return f"Error: {e}", False

def check_wsl_installed():
    """
    Check if WSL (Windows Subsystem for Linux) is installed.
    Docker Desktop requires WSL2.
    """
    try:
        result = subprocess.run(
            ['wsl', '--status'],
            capture_output=True,
            text=True,
            timeout=5
        )
        
        return result.returncode == 0
    except FileNotFoundError:
        return False
    except Exception:
        return False

def check_virtualization():
    """
    Check if virtualization is enabled in BIOS.
    Required for Hyper-V and WSL2.
    """
    try:
        result = subprocess.run(
            ['systeminfo'],
            capture_output=True,
            text=True,
            timeout=15
        )
        
        # Look for Hyper-V Requirements section
        if 'Hyper-V Requirements' in result.stdout:
            # Check if VM Monitor Mode Extensions is Yes
            if 'VM Monitor Mode Extensions: Yes' in result.stdout:
                return True
        
        return False
    except Exception:
        return False

# Run all prerequisite checks
print("=" * 70)
print("DOCKER DESKTOP PREREQUISITES CHECK")
print("=" * 70)

# Windows Edition
os_name, edition_supported = check_windows_edition()
edition_status = "✅ Supported" if edition_supported else "❌ NOT Supported"
print(f"\n1. Windows Edition: {os_name}")
print(f"   Status: {edition_status}")
if not edition_supported:
    print("   ⚠️  Docker Desktop requires Pro, Enterprise, or Education")
    print("   Options: Upgrade Windows or use npm/WSL2 installation")

# WSL Status
wsl_installed = check_wsl_installed()
wsl_status = "✅ Installed" if wsl_installed else "❌ Not Installed"
print(f"\n2. WSL (Windows Subsystem for Linux): {wsl_status}")
if not wsl_installed:
    print("   Action: Run 'wsl --install' in PowerShell as Administrator")

# Virtualization
virt_enabled = check_virtualization()
virt_status = "✅ Enabled" if virt_enabled else "❌ Disabled"
print(f"\n3. Virtualization: {virt_status}")
if not virt_enabled:
    print("   Action: Enable virtualization in BIOS settings")
    print("   How: Restart PC → Enter BIOS → Enable Intel VT-x or AMD-V")

print("\n" + "=" * 70)

# Overall readiness
all_ready = edition_supported and wsl_installed and virt_enabled
if all_ready:
    print("✅ System ready for Docker Desktop installation!")
else:
    print("⚠️  Some prerequisites not met. Address issues above before proceeding.")

print("=" * 70)

## Step 1: Installing WSL2

Docker Desktop on Windows requires WSL2 (Windows Subsystem for Linux 2).

### Installing WSL2

**Open PowerShell as Administrator** and run:

```powershell
wsl --install
```

This command:
- Enables WSL feature
- Installs WSL2
- Installs Ubuntu Linux distribution (default)
- Requires a system restart

### After Restart

After restarting, Ubuntu will launch automatically. You'll be prompted to:
1. Create a Linux username
2. Create a Linux password

**Important**: This is separate from your Windows credentials. Remember this password!

### Verifying WSL2 Installation

```powershell
wsl --list --verbose
```

You should see:
```
  NAME      STATE           VERSION
* Ubuntu    Running         2
```

The VERSION column must show **2** (not 1).

## Step 2: Downloading and Installing Docker Desktop

In [ ]:
# Docker Desktop installation checklist

installation_steps = """
═══════════════════════════════════════════════════════════════
DOCKER DESKTOP INSTALLATION CHECKLIST
═══════════════════════════════════════════════════════════════

STEP 1: Download Docker Desktop
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
□ Visit: https://docs.docker.com/desktop/install/windows-install/
□ Click "Download Docker Desktop for Windows"
□ Save the installer (Docker Desktop Installer.exe)
□ File size: ~500MB - 1GB

STEP 2: Run Installer
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
□ Right-click Docker Desktop Installer.exe
□ Select "Run as Administrator"
□ Wait for Configuration screen to load

STEP 3: Configuration Options
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
CRITICAL: Check these options:
□ ✅ "Use WSL 2 instead of Hyper-V" (REQUIRED)
□ ✅ "Add shortcut to desktop" (optional, but helpful)

STEP 4: Installation
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
□ Click "OK" to start installation
□ Wait 5-15 minutes for installation
□ Click "Close" when finished
□ Restart computer if prompted (usually required)

STEP 5: First Launch
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
□ Find Docker Desktop in Start Menu
□ Launch Docker Desktop
□ Accept Terms of Service
□ Skip survey (or complete if you wish)
□ Wait for "Docker Desktop is running" message
□ Look for whale icon in system tray (bottom-right)

STEP 6: Verify Installation
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
Open PowerShell and run:
□ docker --version
□ docker run hello-world

Expected output:
"Hello from Docker!
This message shows that your installation appears to be working correctly."

═══════════════════════════════════════════════════════════════
"""

print(installation_steps)

## Step 3: Configuring Docker Desktop Resources

Docker Desktop allocates system resources for containers. Let's optimize for n8n.

### Accessing Settings

1. Click Docker whale icon in system tray
2. Click "Settings" (gear icon)
3. Navigate to **Resources**

### Recommended Settings for n8n

**For systems with 8GB RAM:**
- CPUs: 2
- Memory: 3GB
- Swap: 1GB
- Disk image size: 60GB

**For systems with 16GB+ RAM:**
- CPUs: 4
- Memory: 4-6GB
- Swap: 2GB
- Disk image size: 100GB

**Why these settings matter**:
- Too little memory → n8n crashes during large workflow executions
- Too much memory → Windows starves, system becomes sluggish
- Balance is key for laptop deployments

In [ ]:
# Helper to calculate recommended Docker resources

def calculate_docker_resources(total_ram_gb):
    """
    Calculate recommended Docker Desktop resource allocation
    based on total system RAM.
    """
    print("=" * 70)
    print("DOCKER DESKTOP RESOURCE RECOMMENDATIONS")
    print("=" * 70)
    print(f"\nYour Total System RAM: {total_ram_gb}GB")
    print()
    
    if total_ram_gb < 8:
        print("⚠️  WARNING: Less than 8GB RAM")
        print("   Docker Desktop may struggle on this system.")
        print("   Recommendation: Use npm installation instead.")
        print()
        print("If proceeding with Docker:")
        cpus = 2
        memory = 2
        swap = 512
    elif total_ram_gb <= 8:
        cpus = 2
        memory = 3
        swap = 1
    elif total_ram_gb <= 16:
        cpus = 4
        memory = 4
        swap = 2
    else:
        cpus = 4
        memory = 6
        swap = 2
    
    print(f"Recommended Settings:")
    print(f"  CPUs: {cpus} cores")
    print(f"  Memory: {memory}GB")
    print(f"  Swap: {swap}GB")
    print(f"  Disk: 60-100GB")
    print()
    
    remaining_ram = total_ram_gb - memory - (swap / 1024 if swap < 10 else swap)
    print(f"Available for Windows: ~{remaining_ram:.1f}GB")
    
    if remaining_ram < 4:
        print("  ⚠️  Warning: Windows may be sluggish with <4GB remaining")
    
    print("=" * 70)

# Example: Calculate for 16GB system
calculate_docker_resources(16)

print("\n💡 To apply these settings:")
print("   1. Open Docker Desktop")
print("   2. Settings → Resources")
print("   3. Adjust sliders to recommended values")
print("   4. Click 'Apply & Restart'")

## Step 4: Creating Persistent Volume for n8n Data

Docker containers are **stateless** by default - data disappears when you remove the container. Volumes provide **persistent storage**.

### Why Volumes Matter

Without a volume, you'd lose:
- All workflows
- All credentials
- Execution history
- Custom settings

...every time you update or restart the container.

### Creating the Volume

```powershell
docker volume create n8n_data
```

This creates a named volume managed by Docker.

### Verifying the Volume

```powershell
docker volume ls
```

You should see:
```
DRIVER    VOLUME NAME
local     n8n_data
```

### Inspecting Volume Details

```powershell
docker volume inspect n8n_data
```

## Step 5: Running n8n with Docker Run Command

The `docker run` command starts n8n in a container.

### Basic Command

```powershell
docker run -d `
  --name n8n `
  -p 5678:5678 `
  -v n8n_data:/home/node/.n8n `
  docker.n8n.io/n8nio/n8n
```

### Understanding Each Part

- `docker run`: Command to create and start a container
- `-d`: Detached mode (runs in background)
- `--name n8n`: Names the container "n8n" (easier to manage)
- `-p 5678:5678`: Maps port 5678 from container to Windows
- `-v n8n_data:/home/node/.n8n`: Mounts our volume
- `docker.n8n.io/n8nio/n8n`: The official n8n image

### Production-Ready Command

```powershell
docker run -d `
  --name n8n `
  -p 5678:5678 `
  -e GENERIC_TIMEZONE="America/New_York" `
  -e N8N_BASIC_AUTH_ACTIVE=true `
  -e N8N_BASIC_AUTH_USER=admin `
  -e N8N_BASIC_AUTH_PASSWORD=change_this_password `
  -e N8N_ENCRYPTION_KEY=your-random-32-char-key `
  -v n8n_data:/home/node/.n8n `
  --restart unless-stopped `
  docker.n8n.io/n8nio/n8n
```

### New Parameters Explained

- `-e GENERIC_TIMEZONE`: Sets timezone for schedule triggers
- `-e N8N_BASIC_AUTH_*`: Enables authentication (CRITICAL for security)
- `-e N8N_ENCRYPTION_KEY`: Custom encryption key for credentials
- `--restart unless-stopped`: Auto-restart on Docker Desktop startup

In [ ]:
# Docker run command generator

def generate_docker_run_command(
    port=5678,
    timezone="America/New_York",
    auth_user="admin",
    auth_password="change_this_password",
    volume_name="n8n_data"
):
    """
    Generate a customized docker run command for n8n.
    """
    
    command = f'''docker run -d `
  --name n8n `
  -p {port}:{port} `
  -e GENERIC_TIMEZONE="{timezone}" `
  -e N8N_PORT={port} `
  -e N8N_BASIC_AUTH_ACTIVE=true `
  -e N8N_BASIC_AUTH_USER={auth_user} `
  -e N8N_BASIC_AUTH_PASSWORD={auth_password} `
  -v {volume_name}:/home/node/.n8n `
  --restart unless-stopped `
  docker.n8n.io/n8nio/n8n'''
    
    print("=" * 70)
    print("GENERATED DOCKER RUN COMMAND")
    print("=" * 70)
    print()
    print(command)
    print()
    print("=" * 70)
    print()
    print("⚠️  BEFORE RUNNING:")
    print(f"   1. Change password from '{auth_password}' to a strong password")
    print(f"   2. Verify timezone '{timezone}' is correct for your location")
    print(f"   3. Ensure port {port} is not in use")
    print()
    print("📋 To use:")
    print("   1. Copy the command above")
    print("   2. Open PowerShell")
    print("   3. Paste and run")
    print("   4. Wait ~30 seconds for n8n to start")
    print(f"   5. Access at http://localhost:{port}")
    print()
    print("=" * 70)
    
    return command

# Generate with defaults
cmd = generate_docker_run_command(
    timezone="America/New_York",  # Change to your timezone
    auth_password="YourStrongPassword123!"  # Change this!
)

## Step 6: Docker Compose - The Better Way

Docker Compose uses YAML files for configuration, which is:
- Easier to read
- Easier to modify
- Easier to share
- Version controllable

### Creating docker-compose.yml

In [ ]:
# Docker Compose file generator

def generate_docker_compose(
    port=5678,
    timezone="America/New_York",
    auth_user="admin",
    auth_password="change_this_password"
):
    """
    Generate a docker-compose.yml file for n8n.
    """
    
    compose_content = f'''services:
  n8n:
    image: docker.n8n.io/n8nio/n8n:latest
    container_name: n8n
    ports:
      - "{port}:{port}"
    environment:
      - GENERIC_TIMEZONE={timezone}
      - N8N_PORT={port}
      - N8N_BASIC_AUTH_ACTIVE=true
      - N8N_BASIC_AUTH_USER={auth_user}
      - N8N_BASIC_AUTH_PASSWORD={auth_password}
      - N8N_ENCRYPTION_KEY=generate-a-random-32-character-string
      - N8N_LOG_LEVEL=info
    volumes:
      - n8n_data:/home/node/.n8n
    restart: unless-stopped

volumes:
  n8n_data:
'''
    
    print("=" * 70)
    print("DOCKER COMPOSE CONFIGURATION")
    print("=" * 70)
    print()
    print("Save this as: docker-compose.yml")
    print("Recommended location: C:\\n8n-docker\\docker-compose.yml")
    print()
    print("=" * 70)
    print(compose_content)
    print("=" * 70)
    print()
    print("📁 Setup Steps:")
    print("   1. Create folder: C:\\n8n-docker")
    print("   2. Save content above as docker-compose.yml in that folder")
    print("   3. Change N8N_BASIC_AUTH_PASSWORD to a strong password")
    print("   4. Generate encryption key (see below)")
    print()
    print("▶️  To start n8n:")
    print("   cd C:\\n8n-docker")
    print("   docker compose up -d")
    print()
    print("🛑 To stop n8n:")
    print("   docker compose down")
    print()
    print("🔄 To restart n8n:")
    print("   docker compose restart")
    print()
    print("📋 To view logs:")
    print("   docker compose logs -f")
    print()
    print("=" * 70)
    
    return compose_content

# Generate compose file
compose = generate_docker_compose()

## Step 7: Managing Docker Containers

In [ ]:
# Docker management commands reference

import pandas as pd

docker_commands = {
    'Task': [
        'Start n8n container',
        'Stop n8n container',
        'Restart n8n container',
        'View container logs',
        'View live logs (follow)',
        'Check container status',
        'Remove container',
        'Remove container & volume',
        'Update n8n to latest',
        'Access container shell'
    ],
    'Command': [
        'docker start n8n',
        'docker stop n8n',
        'docker restart n8n',
        'docker logs n8n',
        'docker logs -f n8n',
        'docker ps -a',
        'docker rm n8n',
        'docker rm n8n && docker volume rm n8n_data',
        'docker pull n8nio/n8n:latest && docker restart n8n',
        'docker exec -it n8n /bin/sh'
    ],
    'Notes': [
        'Starts existing stopped container',
        'Stops running container (data preserved)',
        'Stop then start',
        'Shows all logs since container creation',
        'Streams logs in real-time (Ctrl+C to exit)',
        'Lists all containers and their status',
        'Deletes container (must stop first)',
        'DANGER: Deletes all data!',
        'Downloads latest version',
        'Opens terminal inside container'
    ]
}

commands_df = pd.DataFrame(docker_commands)

print("=" * 100)
print("DOCKER CONTAINER MANAGEMENT COMMANDS")
print("=" * 100)
print()
print(commands_df.to_string(index=False))
print()
print("=" * 100)

## Step 8: Troubleshooting Common Docker Issues

In [ ]:
# Common Docker issues and solutions

docker_troubleshooting = {
    'Problem': [
        'Error: "no matching manifest for windows"',
        'WSL integration error',
        'Container starts then immediately exits',
        'Cannot access localhost:5678',
        'Docker Desktop won\'t start',
        'Port 5678 already in use',
        'Volume data disappeared'
    ],
    'Cause': [
        'Docker is in Windows container mode',
        'WSL2 not properly configured',
        'Configuration error in environment variables',
        'Port mapping issue or firewall',
        'WSL2 issues or resource conflicts',
        'Another container/app using the port',
        'Used wrong volume or removed it'
    ],
    'Solution': [
        'Right-click Docker → Switch to Linux containers',
        'Settings → Resources → WSL Integration → Enable',
        'Check logs: docker logs n8n',
        'Verify with: docker ps (container should be Up)',
        'Restart WSL: wsl --shutdown, then restart Docker',
        'Change port: -p 8080:5678',
        'Check volumes: docker volume ls'
    ]
}

troubleshooting_df = pd.DataFrame(docker_troubleshooting)

print("=" * 100)
print("DOCKER TROUBLESHOOTING GUIDE")
print("=" * 100)
print()
print(troubleshooting_df.to_string(index=False))
print()
print("=" * 100)
print()
print("💡 General Debugging Steps:")
print("   1. Check Docker Desktop is running (whale icon in system tray)")
print("   2. Run: docker ps -a (check container status)")
print("   3. Run: docker logs n8n (check for errors)")
print("   4. Restart Docker Desktop")
print("   5. Restart WSL2: wsl --shutdown (in PowerShell)")

## Exercises

### Exercise 1: Install and Verify Docker Desktop (Easy)

Complete the Docker Desktop installation and verify it works:

1. Install Docker Desktop following the checklist
2. Run `docker --version` and record the version
3. Run `docker run hello-world` and confirm it works
4. Check Docker Desktop resources (Settings → Resources)
5. Record your current resource allocation

In [ ]:
# Exercise 1: Record your Docker installation

my_docker_install = {
    'Docker Version': '',  # From docker --version
    'hello-world test': '',  # Success or Failed
    'CPUs allocated': 0,  # From Resources settings
    'Memory allocated': 0,  # In GB
    'WSL2 enabled': False,  # True or False
}

# Fill in above, then uncomment:
# print("My Docker Desktop Configuration:")
# for key, value in my_docker_install.items():
#     print(f"{key}: {value}")

### Exercise 2: Deploy n8n with Docker Compose (Medium)

Create a complete n8n deployment using Docker Compose:

1. Create folder `C:\n8n-docker`
2. Create `docker-compose.yml` with your customized settings
3. Set a strong password (not the default!)
4. Set your timezone correctly
5. Run `docker compose up -d`
6. Verify n8n is accessible at http://localhost:5678
7. Test authentication with your username/password

### Exercise 3: Container Management Practice (Hard)

Practice managing your n8n container:

1. Check container status: `docker ps`
2. View last 50 log lines: `docker logs --tail 50 n8n`
3. Stop the container
4. Verify it's stopped with `docker ps -a`
5. Start it again
6. Verify n8n is accessible again
7. Change one environment variable in docker-compose.yml (e.g., log level)
8. Restart the container to apply changes
9. Verify the change took effect in the logs

Document the commands you used for each step.

## Summary

In this module, you learned:

### Installation & Setup

1. **Prerequisites**: Windows Pro/Enterprise, WSL2, virtualization enabled
2. **Docker Desktop**: Installation, configuration, resource allocation
3. **Volumes**: Creating persistent storage for n8n data

### Deployment Methods

4. **docker run**: Direct container creation with command line
5. **Docker Compose**: Declarative YAML-based deployment (recommended)
6. **Environment Variables**: Configuring n8n behavior in containers

### Management

7. **Container Lifecycle**: Start, stop, restart, remove operations
8. **Monitoring**: Viewing logs and container status
9. **Troubleshooting**: Common issues and solutions

### Key Takeaways

- ✅ Docker provides production-ready isolation for n8n
- ✅ WSL2 is required and adds 1-2GB overhead
- ✅ Volumes ensure data persistence across container updates
- ✅ Docker Compose simplifies configuration management
- ✅ `--restart unless-stopped` enables automatic recovery

### Docker vs npm Trade-offs

**Choose Docker when**:
- You need production reliability
- Easy version management is important
- You have Windows Pro/Enterprise
- System has 16GB+ RAM

**Choose npm when**:
- Learning or developing
- On a laptop (battery life matters)
- Have Windows Home
- Want minimal resource usage

### What's Next

- **Module 04**: Initial configuration and first workflow
- **Module 06**: Security best practices for Docker deployments
- **Module 07**: Performance optimization (WSL2 tuning)
- **Module 09**: Update and maintenance procedures

### Additional Resources

- [Docker Desktop Documentation](https://docs.docker.com/desktop/)
- [Docker Compose Reference](https://docs.docker.com/compose/compose-file/)
- [n8n Docker Documentation](https://docs.n8n.io/hosting/installation/docker/)
- [WSL2 Documentation](https://docs.microsoft.com/en-us/windows/wsl/)